# Price Impact Simulator (AgentTorch + Databricks)

This notebook runs the price impact simulator using the AgentTorch framework.
It reads household demographics from a CSV file and runs a single scenario.

The code automatically handles:
- Any number of rows (tested with 1M+ households)
- Flexible column names (automatically matches common variations)
- Missing columns (uses sensible defaults)
- Extra columns (ignored automatically)

Place your CSV file in the same directory, or use the generator cell below to create a synthetic dataset.

In [ ]:
# Install dependencies (Databricks)
# Note: PyPI package is 'agent-torch' but imported as 'agent_torch'
# Upgrade typing_extensions first to fix TypeIs import error
%pip install --upgrade typing_extensions>=4.6.1
%pip install agent-torch pandas numpy torch

# Restart the Python kernel in Databricks if prompted.

In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

# CSV path in current directory
# You can specify any CSV file name here, or place households.csv in this directory
csv_filename = os.environ.get("HOUSEHOLD_CSV", "households.csv")
try:
    csv_path_local = str(Path.cwd() / csv_filename)
except Exception:
    csv_path_local = csv_filename

# Optional: generate a synthetic dataset if the file does not exist
if not os.path.exists(csv_path_local):
    print(f"CSV file not found at {csv_path_local}, generating synthetic data...")
    n_households = 3000
    n_extra = 580

    df = pd.DataFrame({
        "id": np.arange(1, n_households + 1),
        "location": np.random.choice(["north", "south", "east", "west"], n_households),
        "household_size": np.random.choice([1, 2, 3, 4, 5], n_households),
        "num_children": np.random.choice([0, 1, 2, 3], n_households),
        "price_sensitivity": np.random.uniform(0.1, 1.0, n_households),
        "income": np.random.uniform(30000, 150000, n_households),
    })

    for i in range(n_extra):
        df[f"attr_{i+1:03d}"] = np.random.random(n_households)

    df.to_csv(csv_path_local, index=False)
    print(f"Generated synthetic CSV with {n_households} households")
else:
    df_preview = pd.read_csv(csv_path_local, nrows=5)
    print(f"Found CSV file: {csv_path_local}")
    print(f"Columns ({len(df_preview.columns)}): {list(df_preview.columns)[:10]}{'...' if len(df_preview.columns) > 10 else ''}")
    file_size = os.path.getsize(csv_path_local) / (1024 * 1024)
    print(f"File size: {file_size:.2f} MB")

print(f"CSV ready at: {csv_path_local}")

In [ ]:
import sys
from pathlib import Path

# Add current directory to sys.path (works when extracted from tar.gz)
# runner_dbx.py should be in the same directory as this notebook
current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.insert(0, str(current_dir))

# Import runner (will be found in current directory)
from runner_dbx import run_simulation

scenario = {
    "product_category": "new_soft_drink_flavor",
    "new_price": 2.99,
    "baseline_price": 2.49,
}

# Default to CPU (AgentTorch default) - works reliably on Databricks
# Change to "cuda" if GPU is available and you want to use it
device = "cpu"

results = run_simulation(csv_path_local, scenario=scenario, device=device)
results